In [20]:
def __print_shape(x):
    print("{}\n{}".format(np.shape(x), x))
    print("-------------------------------")


def __infer_sparse_weight_tensor(W_conv, k, channels):
    __print_shape(W_conv)
    W_sparse = np.zeros((channels, channels, k, k))
    for ch in range(channels):
        W_sparse[ch][ch] = W_conv[ch]
    W_conv = W_sparse.astype(np.float32)
    __print_shape(W_conv)
    W_matmul = W_conv.transpose(0, 2, 3, 1)
    W_matmul = W_matmul.reshape(channels, channels*k*k)
    __print_shape(W_matmul)
    W_matmul = W_matmul.T
        
k = 2
channels = 3
W_conv = gen_finn_dt_tensor(wdt, (channels, k, k))
W_onnx = __infer_sparse_weight_tensor(W_conv, k, channels) # [k*k*channels, channels]

(3, 2, 2)
[[[-1. -2.]
  [ 5. -6.]]

 [[ 0.  2.]
  [ 1. -7.]]

 [[ 4.  1.]
  [-1. -5.]]]
-------------------------------
(3, 3, 2, 2)
[[[[-1. -2.]
   [ 5. -6.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  2.]
   [ 1. -7.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 4.  1.]
   [-1. -5.]]]]
-------------------------------
(3, 12)
[[-1.  0.  0. -2.  0.  0.  5.  0.  0. -6.  0.  0.]
 [ 0.  0.  0.  0.  2.  0.  0.  1.  0.  0. -7.  0.]
 [ 0.  0.  4.  0.  0.  1.  0.  0. -1.  0.  0. -5.]]
-------------------------------


In [37]:
import numpy as np
from onnx import TensorProto, helper

import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import gen_finn_dt_tensor, calculate_signed_dot_prod_range
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.custom_op.general.multithreshold import multithreshold


# Notes:
# Bipolar datatype not supported/tested
# resType "lut" only tested (because inferVVAU has this attribute fixed)

def __infer_sparse_weight_tensor(W_conv, k_h, k_w, channels):
    #__print_shape(W_conv)
    W_sparse = np.zeros((channels, channels, k_h, k_w))
    for ch in range(channels):
        W_sparse[ch][ch] = W_conv[ch][0]
    W_conv = W_sparse.astype(np.float32)
    #__print_shape(W_conv)
    W_matmul = W_conv.transpose(0, 2, 3, 1)
    W_matmul = W_matmul.reshape(channels, channels*k_h*k_w)
    #__print_shape(W_matmul)
    W_matmul = W_matmul.T
    
    return W_matmul

def  __infer_dense_weight_tensor(W, k, channels):
    """ Weight matrix W has a shape of (k*k*channels, channels). This function
    reverses the process of creating the weight matrix and returns as result a
    dense weight tensor of shape (channels, 1, k, k)
    """
    W = W.T
    W = W.reshape(channels, k, k, channels)
    W = W.transpose(0,3,1,2)
    w_tensor = np.zeros((channels, 1, k, k))
    for ch in range(channels):
        w_tensor[ch][0] = W[ch][ch]
    return w_tensor

def __calculate_dot_prod_range(dt_a, dt_b, len):
    """Returns the (min,max) values a dot product between two unsigned vectors of
    types dt_a and dt_b of len elements can take."""

    min_prod = 2 ** 30
    max_prod = -(2 ** 31)
    for a_val in [dt_a.min(), dt_a.max()]:
        for b_val in [dt_b.min(), dt_b.max()]:
            prod = a_val * b_val * len
            if prod < min_prod:
                min_prod = prod
            if prod > max_prod:
                max_prod = prod
    return (min_prod, max_prod)


def make_single_vvau_modelwrapper(W, pe, k_h, k_w, channels, dim_h, dim_w, wdt, idt, odt, T=None, tdt=None):
    in_shape = [1, dim_h, dim_w, k_h*k_w*channels] # [N, H, W, K*K*CH]
    out_shape = [1, dim_h, dim_w, channels] # [N, H, W, OFM_CH] (OFM_CH=IFM_CH because depthwise convolution)
        
    inp = helper.make_tensor_value_info("inp", TensorProto.FLOAT, in_shape)
    outp = helper.make_tensor_value_info("outp", TensorProto.FLOAT, out_shape)
    
    if T is not None:
        no_act = 0
        node_inp_list = ["inp", "weights", "thresh"]
        actval = odt.min()
    else:
        no_act = 1
        node_inp_list = ["inp", "weights"]
        actval = 0
    
    VVAU_node = helper.make_node(
        "Vector_Vector_Activate_Batch",
        node_inp_list,
        ["outp"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        PE=pe,
        Dim=[dim_h, dim_w],
        Channels=channels,
        Kernel=[k_h, k_w],
        resType="lut",
        ActVal=actval,
        inputDataType=idt.name,
        weightDataType=wdt.name,
        outputDataType=odt.name,
        noActivation=no_act   
    )
    
    graph = helper.make_graph(
        nodes=[VVAU_node],
        name="vvau_graph",
        inputs=[inp],
        outputs=[outp]
    )
    
    model = helper.make_model(graph, producer_name="vvau-model")
    model = ModelWrapper(model)
    
    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)
    model.set_tensor_datatype("weights", wdt)
    
    model.set_initializer("weights", W)
    model.set_tensor_shape("weights", (channels, 1, k_h, k_w))
    
    if T is not None:
        model.set_tensor_datatype("thresh", tdt)
        model.set_initializer("thresh", T)
        
    return model


def prepare_inputs(input_tensor):
    return {"inp": input_tensor}


def test_fpgadataflow_vvau_cppsim(idt, wdt, act, pe, dim, k, channels):
    [dim_h, dim_w] = dim
    [k_h, k_w] = k
    
    if (channels%pe!=0):
        pytest.skip("Requirement Channels divisable by PE is violated.")
    
    W = gen_finn_dt_tensor(wdt, (channels, 1, k_h, k_w)) # [channels, 1, k, k]
    W_onnx = __infer_sparse_weight_tensor(W, k_h, k_w, channels) # [k*k*channels, channels]
    
    x = gen_finn_dt_tensor(idt, (1, dim_h, dim_w, k_h*k_w*channels))
    x_vvau = x.reshape(1, dim_h, dim_w, k_h * k_w, channels // pe, pe)
    x_vvau = x_vvau.transpose(0, 1, 2, 4, 3, 5)
    x_vvau = x_vvau.reshape(1, dim_h, dim_w, channels * k_h * k_w)
            
    if act is None:
        T = None
        tdt = None
        odt = DataType.INT32
    else:
        odt = act
        (min_v, max_v) = __calculate_dot_prod_range(idt, wdt, k_h*k_w*channels)
        n_steps = act.get_num_possible_values()-1
        T = np.random.randint(min_v, max_v-1, (channels, n_steps)).astype(np.float32)
        T = np.sort(T, axis=1)
        tdt = DataType.INT32
        
    model = make_single_vvau_modelwrapper(W, pe, k_h, k_w, channels, dim_h, dim_w, wdt, idt, odt, T, tdt)
    
    model.save("/tmp/test_vvau.onnx")

    model = model.transform(SetExecMode("cppsim"))
    model = model.transform(PrepareCppSim())
    model = model.transform(CompileCppSim())
    model.save("/tmp/test_vvau_compiled.onnx")
    
    input_dict = prepare_inputs(x_vvau)
    
    # Calculate output
    y = np.matmul(x, W_onnx) # y is in [N, H, W, C] format
    if T is not None:
        # Reshape Y, as multithreshold expects y to be in [N, C, H, W] format
        y = np.transpose(y, (0, 3, 1, 2))
        y = multithreshold(y, T)
        y = np.transpose(y, (0, 2, 3, 1))
        y += act.min() 
    y_expected=y
    
    y_produced = oxe.execute_onnx(model, input_dict, return_full_exec_context=False)["outp"]
    
    assert ((y_produced==y_expected).all()), "Cppsim failed!"
    
    return x, x_vvau, W_onnx, W, y_expected, y_produced
    

In [41]:
idt = DataType.UINT4 # DataType.UINT4, DataType.UINT8 (in graph)
wdt = DataType.INT4
#act = DataType.INT4
act = [DataType.UINT4, None]
dim = [14,4]
k = [4,2]
channels = 3
pe = 3

#dim = 5
#k = 3
#channels = 16
#pe = 16
#idt = DataType.UINT4
#act = None
#wdt = DataType.UINT4

# Seems to work when channels=1

x, x_vvau, W_onnx, W, y_expected, y_produced = test_fpgadataflow_vvau_cppsim(idt, wdt, act[0], pe, dim, k, channels)


In [9]:
#print(x)
#print(W_onnx)
print(y_expected)

[[[[ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]]

  [[ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]]

  [[ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]]

  [[ 5. 10.  8.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]]

  [[ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]]

  [[ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [

In [10]:
#print(x)
#print(W)
print(y_produced)

[[[[ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]]

  [[ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]]

  [[ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]]

  [[ 5. 10.  8.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]]

  [[ 5. 11.  7.]
   [ 5. 10.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]]

  [[ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  8.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [ 5. 11.  7.]
   [

In [6]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/test_vvau.onnx")

Serving '/tmp/test_vvau.onnx' at http://0.0.0.0:8081


In [7]:
showInNetron("/tmp/test_vvau_compiled.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/test_vvau_compiled.onnx' at http://0.0.0.0:8081
